In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
import holidays
import datetime as dt
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
os.getcwd()
pd.set_option('display.max_columns', 70)

# CODE

In [2]:
stops_post_covid = pd.read_parquet("~/Downloads/all_stops_ungrouped/all_stops_post_covid.parquet")
df = stops_post_covid
PTCAR = pd.read_csv("./data/PTCAR.csv",sep=";")
df = df.merge(PTCAR,on='REF_PTCAR_NO',how = 'left')
# extract only the HR_ARR values that fall between 6 am and 9 am and 4 pm and 7 pm
df = df[(df['HR_ARR'].dt.hour >= 6) & (df['HR_ARR'].dt.hour <= 9) | (df['HR_ARR'].dt.hour >= 16) & (df['HR_ARR'].dt.hour <= 19)]
df_copy = df
df_copy = df_copy[df_copy['HR_ARR'] != '1900-01-01 00:00:00'].reset_index()
#Any trains arrive within HR_ARR_3min and HR_ARR are behind the the target train
df_copy['HR_ARR_3min'] = df_copy['HR_ARR'] + timedelta(minutes=3)
df_copy = df_copy[['REF_TOD_TRAIN_ID','HR_ARR','HR_ARR_3min','LONG_NAME_NL','TRAIN_DEPARTURE_DATE']]
df_copy = df_copy.reset_index()
df_copy = df_copy.drop(columns='index')
df_copy

,REF_TOD_TRAIN_ID,HR_ARR,HR_ARR_3min,LONG_NAME_NL,TRAIN_DEPARTURE_DATE
0,30539534,2022-06-23 08:11:53,2022-06-23 08:14:53,NaN,2022-06-23
1,30428064,2022-06-05 09:54:22,2022-06-05 09:57:22,ERPS-KWERPS,2022-06-05
2,30428064,2022-06-05 09:59:19,2022-06-05 10:02:19,KORTENBERG,2022-06-05
3,30428064,2022-06-05 09:48:21,2022-06-05 09:51:21,HERENT,2022-06-05
4,30428064,2022-06-05 09:51:59,2022-06-05 09:54:59,VELTEM,2022-06-05
...,...,...,...,...,...
5704400,31636558,2022-12-10 17:57:44,2022-12-10 18:00:44,MOL,2022-12-10
5704401,31329661,2022-10-28 06:13:49,2022-10-28 06:16:49,LEUVEN,2022-10-28
5704402,31325182,2022-10-28 08:28:29,2022-10-28 08:31:29,ARLON,2022-10-28
5704403,31326770,2022-10-28 19:28:05,2022-10-28 19:31:05,LIEGE-GUILLEMINS,2022-10-28


In [22]:
df_copy_2023 = df_copy[df_copy['TRAIN_DEPARTURE_DATE'] == '2023-01-01']
df_copy_2023 = df_copy_2023.reset_index(drop=True)
df_copy_2023

,REF_TOD_TRAIN_ID,HR_ARR,HR_ARR_3min,LONG_NAME_NL,TRAIN_DEPARTURE_DATE
0,31757315,2023-01-01 18:56:40,2023-01-01 18:59:40,BRUSSEL-CENTRAAL,2023-01-01
1,31757436,2023-01-01 08:11:03,2023-01-01 08:14:03,MONS,2023-01-01
2,31756029,2023-01-01 16:23:35,2023-01-01 16:26:35,LIERS,2023-01-01
3,31757584,2023-01-01 19:21:34,2023-01-01 19:24:34,FORRIERES,2023-01-01
4,31757315,2023-01-01 18:41:50,2023-01-01 18:44:50,HAREN-ZUID,2023-01-01
...,...,...,...,...,...
10432,31757599,2023-01-01 18:52:15,2023-01-01 18:55:15,NaN,2023-01-01
10433,31757599,2023-01-01 16:08:19,2023-01-01 16:11:19,NaN,2023-01-01
10434,31757599,2023-01-01 16:49:50,2023-01-01 16:52:50,NaN,2023-01-01
10435,31757599,2023-01-01 19:05:23,2023-01-01 19:08:23,NaN,2023-01-01


In [23]:
df_copy_2023 = df_copy_2023.copy()

def check_overlap(row, other_row):
    return (
        other_row['REF_TOD_TRAIN_ID'] != row['REF_TOD_TRAIN_ID'] and
        other_row['HR_ARR'] >= row['HR_ARR'] and
        other_row['HR_ARR'] <= row['HR_ARR_3min']
    )

unique_names = df_copy_2023['REF_TOD_TRAIN_ID'].unique()

for name in unique_names:
    mask = df_copy_2023['REF_TOD_TRAIN_ID'] == name
    id_rows = [(row['REF_TOD_TRAIN_ID'], row) for _, row in df_copy_2023.loc[mask].iterrows()]
    id_overlaps = {
        ref_id: [other_row['REF_TOD_TRAIN_ID'] for _, other_row in id_rows if check_overlap(row, other_row)]
        for ref_id, row in id_rows
    }
    df_copy_2023.loc[mask, unique_names] = 0

    for ref_id, overlaps in id_overlaps.items():
        for overlap_id in overlaps:
            df_copy_2023.loc[(mask) & (df_copy_2023['REF_TOD_TRAIN_ID'] == overlap_id), ref_id] = 1


In [24]:
df_copy_2023

,REF_TOD_TRAIN_ID,HR_ARR,HR_ARR_3min,LONG_NAME_NL,TRAIN_DEPARTURE_DATE,31757315,31757436,31756029,31757584,31755438,31755301,31755725,31756150,31756590,31756467,31755534,31755509,31756701,31756676,31756748,31755159,31756830,31755033,31756788,31755924,31755747,31757547,31757607,31755399,31757316,31755773,31757372,31755823,31755731,31756036,...,31756881,31757535,31756740,31756527,31756920,31756502,31756851,31755864,31755908,31755140,31757574,31756649,31755957,31756520,31757084,31756531,31757066,31756210,31756430,31755562,31755157,31757550,31755632,31755680,31755647,31756249,31755063,31756924,31757032,31755611,31755450,31755295,31756019,31755400,31757599
0,31757315,2023-01-01 18:56:40,2023-01-01 18:59:40,BRUSSEL-CENTRAAL,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31757436,2023-01-01 08:11:03,2023-01-01 08:14:03,MONS,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31756029,2023-01-01 16:23:35,2023-01-01 16:26:35,LIERS,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31757584,2023-01-01 19:21:34,2023-01-01 19:24:34,FORRIERES,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31757315,2023-01-01 18:41:50,2023-01-01 18:44:50,HAREN-ZUID,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10432,31757599,2023-01-01 18:52:15,2023-01-01 18:55:15,NaN,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10433,31757599,2023-01-01 16:08:19,2023-01-01 16:11:19,NaN,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10434,31757599,2023-01-01 16:49:50,2023-01-01 16:52:50,NaN,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10435,31757599,2023-01-01 19:05:23,2023-01-01 19:08:23,NaN,2023-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_group = df_copy_2023.groupby(['REF_TOD_TRAIN_ID','LONG_NAME_NL']).sum()
df_group = df_group.reset_index()
df_group

/var/folders/53/m33_75cs17xcfmlktknjddb00000gn/T/ipykernel_65815/3744041542.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_group = df_copy_2023.groupby(['REF_TOD_TRAIN_ID','LONG_NAME_NL']).sum()


,REF_TOD_TRAIN_ID,LONG_NAME_NL,31757315,31757436,31756029,31757584,31755438,31755301,31755725,31756150,31756590,31756467,31755534,31755509,31756701,31756676,31756748,31755159,31756830,31755033,31756788,31755924,31755747,31757547,31757607,31755399,31757316,31755773,31757372,31755823,31755731,31756036,31756534,31755365,31756003,...,31756881,31757535,31756740,31756527,31756920,31756502,31756851,31755864,31755908,31755140,31757574,31756649,31755957,31756520,31757084,31756531,31757066,31756210,31756430,31755562,31755157,31757550,31755632,31755680,31755647,31756249,31755063,31756924,31757032,31755611,31755450,31755295,31756019,31755400,31757599
0,31755028,NIVELLES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31755031,AALTER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31755031,ALKEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31755031,BOKRIJK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31755031,BRUGGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10395,31757697,BRUSSEL-CENTRAAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10396,31757697,BRUSSEL-NOORD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10397,31757697,BRUSSEL-ZUID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10398,31757697,BRUSSELS AIRPORT - ZAVENTEM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
df_group['num_ahead'] = df_group.iloc[:, 2:].sum(axis=1)
df_group

,REF_TOD_TRAIN_ID,LONG_NAME_NL,31757315,31757436,31756029,31757584,31755438,31755301,31755725,31756150,31756590,31756467,31755534,31755509,31756701,31756676,31756748,31755159,31756830,31755033,31756788,31755924,31755747,31757547,31757607,31755399,31757316,31755773,31757372,31755823,31755731,31756036,31756534,31755365,31756003,...,31757535,31756740,31756527,31756920,31756502,31756851,31755864,31755908,31755140,31757574,31756649,31755957,31756520,31757084,31756531,31757066,31756210,31756430,31755562,31755157,31757550,31755632,31755680,31755647,31756249,31755063,31756924,31757032,31755611,31755450,31755295,31756019,31755400,31757599,num_ahead
0,31755028,NIVELLES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,31755031,AALTER,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,31755031,ALKEN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,31755031,BOKRIJK,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,31755031,BRUGGE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10395,31757697,BRUSSEL-CENTRAAL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10396,31757697,BRUSSEL-NOORD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10397,31757697,BRUSSEL-ZUID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10398,31757697,BRUSSELS AIRPORT - ZAVENTEM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
df_group['num_ahead'].value_counts()

0.0    10400
Name: num_ahead, dtype: int64

In [10]:
df_group = df_group[['REF_TOD_TRAIN_ID','LONG_NAME_NL','num_ahead']] 
# Pivot the DataFrame to transform LONG_NAME_NL to columns
transformed_df = df_group.pivot(index='REF_TOD_TRAIN_ID', columns='LONG_NAME_NL', values='num_ahead')

# Reset the index to make REF_TOD_TRAIN_ID a column again
transformed_df.reset_index(inplace=True)

# Print the transformed DataFrame
transformed_df

LONG_NAME_NL,REF_TOD_TRAIN_ID,AALST,AALTER,AARSCHOT,ACREN,AISEAU,ALKEN,AMAY,AMPSIN,ANDENNE,ANDERLECHT,ANGLEUR,ANS,ANSEREMME,ANTOING,ANTWERPEN-BERCHEM,ANTWERPEN-CENTRAAL,ANTWERPEN-LUCHTBAL,ANTWERPEN-NOORDERDOKKEN,ANTWERPEN-ZUID,APPELTERRE,ARCHENNES,ARLON,ASSE,ASSESSE,ATH,ATHUS,AUBANGE,AUVELAIS,AYE,AYWAILLE,BALEGEM-DORP,BALEGEM-ZUID,BALEN,BARVAUX,...,VORST-ZUID,WAARSCHOOT,WALCOURT,WAREGEM,WAREMME,WATERLOO,WATERMAAL,WAVRE,WEERDE,WELKENRAEDT,WELLE,WERVIK,WESPELAAR-TILDONK,WETTEREN,WEVELGEM,WEZEMAAL,WICHELEN,WIJGMAAL,WILDERT,WILLEBROEK,WONDELGEM,YVES-GOMEZEE,YVOIR,ZANDBERGEN,ZAVENTEM,ZEDELGEM,ZEEBRUGGE-STRAND,ZELE,ZELLIK,ZICHEM,ZINGEM,ZOLDER,ZONHOVEN,ZOTTEGEM,ZWIJNDRECHT
0,31755028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31755031,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31755032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31755033,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31755043,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,31757684,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1129,31757686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1130,31757688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1131,31757696,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_group

,REF_TOD_TRAIN_ID,LONG_NAME_NL,num_ahead
0,31755028,NIVELLES,1.0
1,31755031,AALTER,1.0
2,31755031,ALKEN,1.0
3,31755031,BOKRIJK,1.0
4,31755031,BRUGGE,1.0
...,...,...,...
10395,31757697,BRUSSEL-CENTRAAL,1.0
10396,31757697,BRUSSEL-NOORD,1.0
10397,31757697,BRUSSEL-ZUID,1.0
10398,31757697,BRUSSELS AIRPORT - ZAVENTEM,1.0


# Loop through each unique REF_PTCAR_NO
for name in df_copy_2023['REF_TOD_TRAIN_ID'].unique():
    mask = df_copy_2023['REF_TOD_TRAIN_ID'] == name
    id_rows = [(row['REF_TOD_TRAIN_ID'], row) for _, row in df_copy_2023.loc[mask].iterrows()]
    # Create a dictionary of REF_TOD_TRAIN_IDs and their overlapping IDs
    id_overlaps = {ref_id: [other_row['REF_TOD_TRAIN_ID'] for _, other_row in id_rows if ref_id != other_row['REF_TOD_TRAIN_ID'] and other_row['HR_ARR'] >= row['HR_ARR'] and other_row['HR_ARR'] <= row['HR_ARR_3min']] for ref_id, row in id_rows}
    
    for ref_id in df_copy['REF_TOD_TRAIN_ID'].unique():
        df_copy_2023.loc[mask, ref_id] = 0
    
    for ref_id, overlaps in id_overlaps.items():
        for overlap_id in overlaps:
            df_copy_2023.loc[(mask) & (df_copy_2023['REF_TOD_TRAIN_ID'] == overlap_id), ref_id] = 1